# Patcher

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image

In [2]:
root_dir = '../../Textures'

In [ ]:
folders_to_patch = [
    '2D_Municipio', 
    '2D_Palazzo_Maresa', 
    '2D_Rilievi_Centro_Italia', 
    'Testures_Tiled_4096_SantElia_Rudere_CentroItalia', 
    'Testures_Tiled_4096_Torre_Fontecchio_CentroItalia', 
    'Textures_Tiled_4096_Corvara Torre_CentroItalia', 
    'Textures_Tiled_4096_Municipio_Tredozio', 
    'Textures_Tiled_4096_Palazzo_Fantini_Tredozio', 
    'Textures_Tiled_4096_Palazzo_Maresa_Tredozio', 
    'Textures_Tiled_4096_Piazza_Tredozio', 
    'Textures_Tiled_4096_PiccoloRudere_Corvara_CentroItalia', 
    'Textures_Tiled_4096_Poggio di Roio-Ed3_CentroItalia', 
    'Textures_Tiled_4096_Ripa_Chiesetta_CentroItalia', 
    'Textures_Tiled_4096_Scuola_Tredozio', 
    'Texture_Tiled_4096_ChiesaSconsacrata_Navelli_CentroItalia', 
    'Texture_Tiled_4096_SantaMariadelMontePalazzina_CentroItalia', 
    'Texture_Tiled_4096_SantaMariadelPonte_Chiesa_CentroItalia'
]

In [ ]:
# folders_to_patch = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
print(folders_to_patch)

['Testures_Tiled_4096_SantElia_Rudere_CentroItalia', 'Testures_Tiled_4096_Torre_Fontecchio_CentroItalia', 'Textures_Tiled_4096_Corvara Torre_CentroItalia', 'Textures_Tiled_4096_Municipio_Tredozio', 'Textures_Tiled_4096_Palazzo_Fantini_Tredozio', 'Textures_Tiled_4096_PiccoloRudere_Corvara_CentroItalia', 'Textures_Tiled_4096_Poggio di Roio-Ed3_CentroItalia', 'Textures_Tiled_4096_Ripa_Chiesetta_CentroItalia', 'Textures_Tiled_4096_Scuola_Tredozio', 'Texture_Tiled_4096_ChiesaSconsacrata_Navelli_CentroItalia', 'Texture_Tiled_4096_SanNicandroCasolare_CentroItalia', 'Texture_Tiled_4096_SantaMariadelMontePalazzina_CentroItalia', 'Texture_Tiled_4096_SantaMariadelPonte_Chiesa_CentroItalia']


## Funzioni

In [4]:
def crop_cv2_patches(data_path, patch_size=512):
    images_dir = os.path.join(data_path, "imgs")
    labels_dir = os.path.join(data_path, "labels")

    patches_dir = os.path.join(data_path, "patches")
    os.makedirs(patches_dir, exist_ok=True)

    img_dir = os.path.join(patches_dir, "imgs")
    label_dir = os.path.join(patches_dir, "labels")
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for image_file in os.listdir(images_dir):

        image_path = os.path.join(images_dir, image_file)
        label_file = os.path.splitext(image_file)[0] + '.png'
        label_path = os.path.join(labels_dir, label_file)

        if not os.path.isfile(image_path):
            print(f"Immagine non trovata: {image_path}")
            continue
        if not os.path.isfile(label_path):
            print(f"Label non trovata: {label_path}")
            continue

        img = cv2.imread(image_path)
        label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)

        h, w, _ = img.shape

        new_h = int(np.ceil(h / patch_size) * patch_size)
        new_w = int(np.ceil(w / patch_size) * patch_size)

        img_resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        label_resized = cv2.resize(label, (new_w, new_h), interpolation=cv2.INTER_NEAREST)

        base_name = os.path.splitext(image_file)[0]
        count = 0
        for y in range(0, new_h, patch_size):
            for x in range(0, new_w, patch_size):
                img_patch = img_resized[y:y+patch_size, x:x+patch_size]
                label_patch = label_resized[y:y+patch_size, x:x+patch_size]

                img_patch_path = os.path.join(img_dir, f"{base_name}_patch_{count:04d}.jpg")
                label_patch_path = os.path.join(label_dir, f"{base_name}_patch_{count:04d}.png")

                cv2.imwrite(img_patch_path, img_patch)
                cv2.imwrite(label_patch_path, label_patch)

                count += 1

        print(f"Saved {count} patches to '{img_dir}' and '{label_dir}'")

In [ ]:
def check_labels(label_dir):
    for filename in os.listdir(label_dir):
        path = os.path.join(label_dir, filename)
        if not os.path.isfile(path):
            continue
        label = Image.open(path)
        label_array = np.array(label)

        unique_values = np.unique(label_array)
        if not set(unique_values).issubset({0, 255}):
            print(f"❌ {filename} has different values from 0 and 255: {unique_values}")

In [ ]:
def show_image_label_patches(patch_dir):
    img_dir = os.path.join(patch_dir, "images")
    label_dir = os.path.join(patch_dir, "labels")

    img_files = sorted(os.listdir(img_dir))
    label_files = sorted(os.listdir(label_dir))

    assert len(img_files) == len(label_files), "Mismatch in image and label counts"

    n = len(img_files)
    rows = n

    plt.figure(figsize=(10, rows * 2.5))
    for i in range(n):
        img_path = os.path.join(img_dir, img_files[i])
        label_path = os.path.join(label_dir, label_files[i])

        img = cv2.imread(img_path)
        label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.subplot(rows, 2, i * 2 + 1)
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(f'Image {i + 1}')

        plt.subplot(rows, 2, i * 2 + 2)
        cmap = 'gray' if len(label.shape) == 2 or (len(label.shape) == 3 and label.shape[2] == 1) else None
        plt.imshow(label, cmap=cmap)
        plt.axis('off')
        plt.title(f'Label {i + 1}')

    plt.tight_layout()
    plt.show()

## Patcher

In [ ]:
for folder in folders_to_patch:
    print(f"Processing folder: {folder}")
    data_folder = os.path.join(root_dir, folder)        
    crop_cv2_patches(data_folder)

In [8]:
for folder in folders_to_patch:
    out_dir = os.path.join(root_dir, folder, "patches", "labels")
    check_labels(out_dir)